1\. Write a function that converts number representation (bin<->dec<->hex)

In [22]:
def conversion(x,curr_base=10):
    if curr_base==10:
        decx = x
        binx = bin(x)
        hexx = hex(x)
        print("The inserted number is", decx)
        print("Its binary rappresentation is", binx)
        print("Its hexadecimal rappresentation is", hexx)
    elif curr_base==2:
        binx = bin(int(str(x),2))
        decx = int(binx,2)
        hexx = hex(decx)
        print("The inserted number is", binx)
        print("Its decimal rappresentation is", decx)
        print("Its hexadecimal rappresentation is", hexx)
    else:
        hexx = hex(int(str(x),16))
        decx = int(hexx,16)
        binx = bin(decx)
        print("The inserted number is", x)
        print("Its binary rappresentation is", binx)
        print("Its decimal rappresentation is", hexx)
    return binx,decx,hexx

        
x = input("Insert an integer ")
curr_base = int(input("Inserire la base "))
conversion(x,curr_base)


Insert an integer a9
Inserire la base 16
The inserted number is a9
Its binary rappresentation is 0b10101001
Its decimal rappresentation is 0xa9


('0b10101001', 169, '0xa9')

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [1]:
def conv_into_float(x):
    word = list(str(bin(x)))
    while(len(word)<34):
        word.insert(2,'0')
    #print(word)
    exponent = ""
    mantissa = ""
    for x in range(3,11):
        exponent += word[x]
    exponent = int(exponent,2)
    #print(exponent)
    for x in range(11,34):
        mantissa += word[x]
    #print(mantissa)
    frac = 0
    for x in range(len(mantissa)):
        frac += int(mantissa[x])*2**(-(x+1))
    #print(frac)
    floatnum = (1+frac)*2**(exponent-127)
    if int(word[2])==1:
        floatnum = -floatnum
    return floatnum


#x = 0xbfa00000
x = 0xc0c00000
print(conv_into_float(x))

-6.0


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [12]:
import math
import sys

overx = 1.
underx = 1.

while((underx/2)>0):
    underx = underx/2
print("The minimum value is", underx)
print(sys.float_info)

while((overx*2)<math.inf):
    overx = overx*2
print("The max value is", overx)

The minimum value is 5e-324
sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)
The max value is 8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [8]:
initial_value = 1.
step = 1/10

while initial_value != (initial_value + step):
    step = step / 10

print(step)

1.0000000000000001e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [19]:
import math as m
def quadratic_sol(a,b,c):
    r_delta = m.sqrt(b**2-4*a*c)
    x1 = (-b + r_delta)/(2*a)
    x2 = (-b - r_delta)/(2*a)
    return x1,x2
'''    print(b**2-4*a*c)
    print(r_delta**2)
    print(r_delta)
    print(r_delta/2/a)   '''

# The square root in r_delta makes the result no more precise. In particular r_delta is slightly bigger than what that value should be.
#  The error propagates through the successive operations. That's why multiplying and dividing by the opposite of the numerator 
#  we obtain a formula which is mathematically equivalent but, when implemented, gives completely different results compared to
#  to the initial one.
def modified_quadratic_sol(a,b,c):
    r_delta = m.sqrt(b**2-4*a*c)
    
    x1 = (4*a*c)/(2*a*(-b-r_delta))
    x2 = (4*a*c)/(2*a*(-b+r_delta))
    return x1,x2

def good_quadratic_sol(a,b,c):
    x1 = (-b/(2*a)+m.e**((0.5*m.log(b**2-4*a*c))-m.log(2*a)))
    x2 = (-b/(2*a)-m.e**((0.5*m.log(b**2-4*a*c))-m.log(2*a)))
    return x1,x2
'''    print(m.e**(0.5*m.log(b**2-4*a*c)))
    print((m.e**(0.5*m.log(b**2-4*a*c)*2)))
    print(+m.e**((0.5*m.log(b**2-4*a*c))-m.log(2*a)))   '''

# sqrt(b**2-4*a*c) < r_delta

a = 0.001
b = 1000
c = 0.001
print(quadratic_sol(a,b,c))
print(modified_quadratic_sol(a,b,c))
print(good_quadratic_sol(a,b,c))

(-9.999894245993346e-07, -999999.999999)
(-1.0000000000009999e-06, -1000010.5755125057)
(-1.0007061064243317e-06, -999999.9999989993)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [20]:
def f(x):
    return x*(x-1)

x=1
delta = 1e-2
def_der = (f(x+delta) - f(x))/delta
print(def_der)
# The values do not perfectly match because using the definition of derivative, with delta = 10^-2, we're not
#  mathematically using the limit, we're instead using an approximation of what it is the limit.
# Computing analitically the derivative gives indeed perfectly 1.

deltas = [10**(-2*i) for i in range(2,8)]
#print(deltas)
derivatives = [(f(x+delt) - f(x))/delt for delt in deltas]
print(derivatives)
# The accuracy increases until delta reaches the value 10^-14, from that on the accuracy gets worse

1.010000000000001
[1.0000999999998899, 1.0000009999177333, 1.0000000039225287, 1.000000082840371, 1.0000889005833413, 0.9992007221626509]


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [7]:
import math as m
import numpy as np
def semicircle_integral(N):
    h = 2/N
    sliced_domain = np.linspace(-1,1,N)
    semicircle = lambda x: m.sqrt(1-x**2)
    y = [semicircle(x) for x in sliced_domain]
    return h*sum(y)

print(semicircle_integral(100))
print(semicircle_integral(10000))
print(semicircle_integral(100000))
print(semicircle_integral(1000000))

%timeit semicircle_integral(1000000)
# N can be increased for up to almost 1 million before the function takes more than a second to get the result
# Watching the execution time for different Ns it seems that the time grows linearly with N. On the other hand, as it is
# showed, letting N grow by orders of magnitude gets a difference in terms of fractions of thousandths

1.553417929404895
1.5706375839994526
1.5707805662399095
1.5707947543354452
1.05 s ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
